In [2]:
import yfinance as yf
import ta
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import numpy as np
import hvplot.pandas

C:\Users\jacob\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class pricecross(Strategy):

    n1 = 2

    def init(self):
        close = self.data.Close
        self.sma = self.I(ta.trend.sma_indicator, pd.Series(close), self.n1)

    def next(self):
        if crossover(df['Close'], self.sma):
            self.buy()
        elif crossover(self.sma, df['Close']):
            self.sell()

In [4]:
df = yf.download('ETH-USD', interval='1m', period='7D')

[*********************100%%**********************]  1 of 1 completed


In [5]:
df.head()

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-01-03 00:01:00+00:00,2356.685059,2356.685059,2356.685059,2356.685059,2356.685059,0
2024-01-03 00:03:00+00:00,2358.718994,2358.718994,2358.718994,2358.718994,2358.718994,0
2024-01-03 00:04:00+00:00,2359.686523,2359.686523,2359.686523,2359.686523,2359.686523,0
2024-01-03 00:05:00+00:00,2358.689209,2358.689209,2358.689209,2358.689209,2358.689209,0
2024-01-03 00:06:00+00:00,2357.551025,2357.551025,2357.551025,2357.551025,2357.551025,2016256


In [6]:
bt = Backtest(df, pricecross, cash=100000, commission=0.00,
exclusive_orders=True)

output = bt.run()

In [7]:
output

Start                     2024-01-03 00:01...
End                       2024-01-09 00:04...
Duration                      6 days 00:03:00
Exposure Time [%]                   91.886725
Equity Final [$]                100500.556396
Equity Peak [$]                 100963.457031
Return [%]                           0.500556
Buy & Hold Return [%]               -1.005596
Return (Ann.) [%]                 1857.786901
Volatility (Ann.) [%]              991.065313
Sharpe Ratio                         1.874535
Sortino Ratio                      167.677568
Calmar Ratio                       203.106664
Max. Drawdown [%]                   -9.146854
Avg. Drawdown [%]                   -1.149908
Max. Drawdown Duration        5 days 06:46:00
Avg. Drawdown Duration        0 days 12:02:00
# Trades                                   20
Win Rate [%]                             60.0
Best Trade [%]                       0.465985
Worst Trade [%]                     -0.156751
Avg. Trade [%]                    

In [8]:
optim = bt.optimize(n1 = range(2,6,4),
    maximize= 'Return [%]')

  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
optim

Start                     2024-01-03 00:01...
End                       2024-01-09 00:04...
Duration                      6 days 00:03:00
Exposure Time [%]                   91.886725
Equity Final [$]                100500.556396
Equity Peak [$]                 100963.457031
Return [%]                           0.500556
Buy & Hold Return [%]               -1.005596
Return (Ann.) [%]                 1857.786901
Volatility (Ann.) [%]              991.065313
Sharpe Ratio                         1.874535
Sortino Ratio                      167.677568
Calmar Ratio                       203.106664
Max. Drawdown [%]                   -9.146854
Avg. Drawdown [%]                   -1.149908
Max. Drawdown Duration        5 days 06:46:00
Avg. Drawdown Duration        0 days 12:02:00
# Trades                                   20
Win Rate [%]                             60.0
Best Trade [%]                       0.465985
Worst Trade [%]                     -0.156751
Avg. Trade [%]                    

In [10]:
#build interactive backtest graph

signals_df = df

window = 2

signals_df.head()

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-01-03 00:01:00+00:00,2356.685059,2356.685059,2356.685059,2356.685059,2356.685059,0
2024-01-03 00:03:00+00:00,2358.718994,2358.718994,2358.718994,2358.718994,2358.718994,0
2024-01-03 00:04:00+00:00,2359.686523,2359.686523,2359.686523,2359.686523,2359.686523,0
2024-01-03 00:05:00+00:00,2358.689209,2358.689209,2358.689209,2358.689209,2358.689209,0
2024-01-03 00:06:00+00:00,2357.551025,2357.551025,2357.551025,2357.551025,2357.551025,2016256


In [11]:
signals_df = df.loc[:,["Close"]]

In [12]:
window = 2

In [13]:
signals_df['MA'] = signals_df['Close'].rolling(window=window).mean()

signals_df['Signal'] = 0.0


signals_df['Signal'][window:] = np.where(
    signals_df['Close'][window:] > signals_df['MA'][window:], 1.0, 0.0
)

In [14]:
signals_df['Entry/Exit'] = signals_df['Signal'].diff()

signals_df.tail(10)

,Close,MA,Signal,Entry/Exit
Datetime,,,,
2024-01-08 23:54:00+00:00,2332.331055,2332.364014,0.0,-1.0
2024-01-08 23:55:00+00:00,2332.550293,2332.440674,1.0,1.0
2024-01-08 23:56:00+00:00,2333.166748,2332.858521,1.0,0.0
2024-01-08 23:57:00+00:00,2333.346680,2333.256714,1.0,0.0
2024-01-08 23:58:00+00:00,2333.706055,2333.526367,1.0,0.0
2024-01-09 00:00:00+00:00,2332.868164,2333.287109,0.0,-1.0
2024-01-09 00:01:00+00:00,2333.000977,2332.934570,1.0,1.0
2024-01-09 00:02:00+00:00,2331.766846,2332.383911,0.0,-1.0
2024-01-09 00:03:00+00:00,2331.770020,2331.768433,1.0,1.0


In [15]:
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Close'].hvplot.scatter(
    color='yellow',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Close'].hvplot.scatter(
    color='purple',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

security_close = signals_df[['Close']].hvplot(
    line_color='blue',
    ylabel='Price in $',
    width=1000,
    height=400
)


moving_avgs = signals_df[['MA']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)


entry_exit_plot = security_close * moving_avgs * entry * exit

entry_exit_plot.opts(
    title="ETH - MA, Entry and Exit Points"
)

:Overlay
   .Curve.I          :Curve   [Datetime]   (Close)
   .Curve.II         :Curve   [Datetime]   (MA)
   .Scatter.Close.I  :Scatter   [Datetime]   (Close)
   .Scatter.Close.II :Scatter   [Datetime]   (Close)

In [16]:
# Build performance table
initial_capital = float(100000)
share_size = 42
# columns
signals_df['Position'] = share_size * signals_df['Signal']
signals_df['Entry/Exit Position'] = signals_df['Position'].diff()
signals_df['Portfolio Holdings'] = signals_df['Close'] * signals_df['Position']
signals_df['Portfolio Cash'] = initial_capital - (signals_df['Close'] * signals_df['Entry/Exit Position']).cumsum() 
signals_df['Portfolio Total'] = signals_df['Portfolio Cash'] + signals_df['Portfolio Holdings']
signals_df['Portfolio Daily Returns'] = signals_df['Portfolio Total'].pct_change()
signals_df['buy and hold'] = signals_df['Close'] * 42
signals_df.tail()

,Close,MA,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,buy and hold
Datetime,,,,,,,,,,,
2024-01-09 00:00:00+00:00,2332.868164,2333.287109,0.0,-1.0,0.0,-42.0,0.000000,122728.398438,122728.398438,-0.000287,97980.462891
2024-01-09 00:01:00+00:00,2333.000977,2332.934570,1.0,1.0,42.0,42.0,97986.041016,24742.357422,122728.398438,0.000000,97986.041016
2024-01-09 00:02:00+00:00,2331.766846,2332.383911,0.0,-1.0,0.0,-42.0,0.000000,122676.564941,122676.564941,-0.000422,97934.207520
2024-01-09 00:03:00+00:00,2331.770020,2331.768433,1.0,1.0,42.0,42.0,97934.340820,24742.224121,122676.564941,0.000000,97934.340820
2024-01-09 00:04:00+00:00,2332.986328,2332.378174,1.0,0.0,42.0,0.0,97985.425781,24742.224121,122727.649902,0.000416,97985.425781


In [17]:
# build graph
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='yellow',
    marker='v',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='purple',
    marker='^',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    xlabel='Date',
    width=1000,
    height=400
)

portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(
    title="ETH Algorithm - Total Portfolio Value",
    yformatter='%.0f'
)

:Overlay
   .Curve.I                    :Curve   [Datetime]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Datetime]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [Datetime]   (Portfolio Total)

In [18]:
portfolio_total = signals_df['Portfolio Total'].hvplot(
    line_color='gray',
    ylabel='Total Portfolio Value',
    xlabel='Date',
    width=1000,
    height=400
)
closing = signals_df['buy and hold'].hvplot(
    line_color='purple',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

performance = portfolio_total * closing

performance.opts(
    title="ETH Algorithm - Total Portfolio Value",
    yformatter='%.0f'
)

:Overlay
   .Curve.Portfolio_Total :Curve   [Datetime]   (Portfolio Total)
   .Curve.Buy_and_hold    :Curve   [Datetime]   (buy and hold)

In [19]:
signals_df.describe()

,Close,MA,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,buy and hold
count,8369.000000,8368.000000,8369.000000,8368.000000,8369.000000,8368.000000,8369.000000,8368.000000,8368.000000,8367.000000,8369.000000
mean,2254.920203,2254.909458,0.498267,0.000120,20.927231,0.005019,47200.028752,61384.751429,108590.420720,0.000025,94706.648543
std,44.213408,44.191979,0.500027,0.678777,21.001129,28.508646,47385.119408,47692.957793,6293.785809,0.000524,1856.963153
min,2113.925293,2120.409302,0.000000,-1.000000,0.000000,-42.000000,0.000000,113.233398,97142.461914,-0.024155,88784.862305
25%,2231.217529,2231.373749,0.000000,0.000000,0.000000,0.000000,0.000000,14433.454346,102955.773071,0.000000,93711.136230
50%,2243.501465,2243.540649,0.000000,0.000000,0.000000,0.000000,0.000000,98383.307617,108521.713867,0.000000,94227.061523
75%,2262.774902,2262.711548,1.000000,0.000000,42.000000,0.000000,94232.219238,108482.405518,113038.549316,0.000027,95036.545898
max,2385.117676,2384.310791,1.000000,1.000000,42.000000,42.000000,100174.942383,122825.554199,122869.615234,0.011552,100174.942383
